In [1]:
#basic imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
warnings.filterwarnings('ignore')

In [3]:
device = torch.device("mps")

In [4]:
df = pd.read_csv('weatherAUS-2.csv')

In [5]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
MinTemp,143975.0,12.194034,6.398495,-8.5,7.6,12.0,16.9,33.9
MaxTemp,144199.0,23.221348,7.119049,-4.8,17.9,22.6,28.2,48.1
Rainfall,142199.0,2.360918,8.478060,0.0,0.0,0.0,0.8,371.0
Evaporation,82670.0,5.468232,4.193704,0.0,2.6,4.8,7.4,145.0
Sunshine,75625.0,7.611178,3.785483,0.0,4.8,8.4,10.6,14.5
WindGustSpeed,135197.0,40.035230,13.607062,6.0,31.0,39.0,48.0,135.0
WindSpeed9am,143693.0,14.043426,8.915375,0.0,7.0,13.0,19.0,130.0
WindSpeed3pm,142398.0,18.662657,8.809800,0.0,13.0,19.0,24.0,87.0
Humidity9am,142806.0,68.880831,19.029164,0.0,57.0,70.0,83.0,100.0
Humidity3pm,140953.0,51.539116,20.795902,0.0,37.0,52.0,66.0,100.0


In [8]:
from ydata_profiling import ProfileReport
report = ProfileReport(df)
#report

In [9]:
#dropping rows where target is null
df.dropna(subset=['RainTomorrow','Rainfall','RainToday'], inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140787 entries, 0 to 145458
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           140787 non-null  object 
 1   Location       140787 non-null  object 
 2   MinTemp        140319 non-null  float64
 3   MaxTemp        140480 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81093 non-null   float64
 6   Sunshine       73982 non-null   float64
 7   WindGustDir    131624 non-null  object 
 8   WindGustSpeed  131682 non-null  float64
 9   WindDir9am     131127 non-null  object 
 10  WindDir3pm     137117 non-null  object 
 11  WindSpeed9am   139732 non-null  float64
 12  WindSpeed3pm   138256 non-null  float64
 13  Humidity9am    139270 non-null  float64
 14  Humidity3pm    137286 non-null  float64
 15  Pressure9am    127044 non-null  float64
 16  Pressure3pm    127018 non-null  float64
 17  Cloud9am       88162 non-null   fl

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140787 entries, 0 to 145458
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           140787 non-null  object 
 1   Location       140787 non-null  object 
 2   MinTemp        140319 non-null  float64
 3   MaxTemp        140480 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81093 non-null   float64
 6   Sunshine       73982 non-null   float64
 7   WindGustDir    131624 non-null  object 
 8   WindGustSpeed  131682 non-null  float64
 9   WindDir9am     131127 non-null  object 
 10  WindDir3pm     137117 non-null  object 
 11  WindSpeed9am   139732 non-null  float64
 12  WindSpeed3pm   138256 non-null  float64
 13  Humidity9am    139270 non-null  float64
 14  Humidity3pm    137286 non-null  float64
 15  Pressure9am    127044 non-null  float64
 16  Pressure3pm    127018 non-null  float64
 17  Cloud9am       88162 non-null   fl

In [13]:
#missing 
(df.isnull().sum().sort_values(ascending=False)*100)/len(df)

Sunshine         47.451114
Evaporation      42.400222
Cloud3pm         39.843167
Cloud9am         37.379161
Pressure3pm       9.780022
Pressure9am       9.761555
WindDir9am        6.861429
WindGustDir       6.508413
WindGustSpeed     6.467216
WindDir3pm        2.606775
Humidity3pm       2.486735
Temp3pm           1.863808
WindSpeed3pm      1.797751
Humidity9am       1.077514
WindSpeed9am      0.749359
Temp9am           0.465952
MinTemp           0.332417
MaxTemp           0.218060
RainToday         0.000000
Date              0.000000
Location          0.000000
Rainfall          0.000000
RainTomorrow      0.000000
dtype: float64

In [14]:
drop_col = ['Sunshine','Evaporation','Cloud3pm','Cloud9am']
df.drop(drop_col,axis=1,inplace=True)

In [15]:
(df.isnull().sum().sort_values(ascending=False)*100)/len(df)

Pressure3pm      9.780022
Pressure9am      9.761555
WindDir9am       6.861429
WindGustDir      6.508413
WindGustSpeed    6.467216
WindDir3pm       2.606775
Humidity3pm      2.486735
Temp3pm          1.863808
WindSpeed3pm     1.797751
Humidity9am      1.077514
WindSpeed9am     0.749359
Temp9am          0.465952
MinTemp          0.332417
MaxTemp          0.218060
RainToday        0.000000
Date             0.000000
Location         0.000000
Rainfall         0.000000
RainTomorrow     0.000000
dtype: float64

In [16]:
df.groupby('Location').count()[['Pressure9am','Pressure3pm','WindGustSpeed','WindDir9am']]

,Pressure9am,Pressure3pm,WindGustSpeed,WindDir9am
Location,,,,
Adelaide,3014,3013,3000,2772
Albany,2971,2972,0,2772
Albury,2991,2987,2983,2479
AliceSprings,3024,3022,2976,2796
BadgerysCreek,2749,2734,2864,2508
Ballarat,2993,2994,2994,2926
Bendigo,3016,3021,2995,2787
Brisbane,3132,3130,3120,3065
Cairns,2964,2964,2947,2922


In [17]:
df = df[(df['Location'] != 'MountGinini') & (df['Location'] != 'Newcastle') & (df['Location'] != 'Penrith') & (df['Location'] != 'SalmonGums')]

In [18]:
(df.isnull().sum().sort_values(ascending=False)*100)/len(df)

WindDir9am       5.924607
WindGustDir      4.644322
WindGustSpeed    4.599427
Humidity3pm      1.841474
WindDir3pm       1.702918
Pressure3pm      1.681245
Pressure9am      1.661119
Temp3pm          1.216038
WindSpeed3pm     1.160307
Humidity9am      0.772506
WindSpeed9am     0.592151
Temp9am          0.164099
MinTemp          0.139330
MaxTemp          0.103723
RainToday        0.000000
Date             0.000000
Location         0.000000
Rainfall         0.000000
RainTomorrow     0.000000
dtype: float64

In [19]:
location_wise_pressure = df.groupby('Location')[['Pressure9am','Pressure3pm']].mean()

In [20]:
list(location_wise_pressure.loc['Albury'])

[1018.3605817452357, 1015.7476062939404]

In [21]:
def fill_pressure(cols):
    p_9am = cols[0]
    p_3pm = cols[1]
    location = cols[2]
    if (pd.isna(p_9am) == True) and (pd.isna(p_3pm) == False):
        return [p_3pm,p_3pm]
    elif (pd.isna(p_9am) == False) and (pd.isna(p_3pm) == True):
        return [p_9am,p_9am]
    elif (pd.isna(p_9am) == True) and (pd.isna(p_3pm) == True):
        return list(location_wise_pressure.loc[location])
    else:
        return [p_9am,p_3pm]
    

In [22]:
df[['Pressure9am','Pressure3pm']] = df[['Pressure9am','Pressure3pm','Location']].apply(fill_pressure,axis=1,result_type='expand')

In [23]:
(df.isnull().sum().sort_values(ascending=False)*100)/len(df)

WindDir9am       5.924607
WindGustDir      4.644322
WindGustSpeed    4.599427
Humidity3pm      1.841474
WindDir3pm       1.702918
Temp3pm          1.216038
WindSpeed3pm     1.160307
Humidity9am      0.772506
WindSpeed9am     0.592151
Temp9am          0.164099
MinTemp          0.139330
MaxTemp          0.103723
Pressure3pm      0.000000
RainToday        0.000000
Date             0.000000
Pressure9am      0.000000
Location         0.000000
Rainfall         0.000000
RainTomorrow     0.000000
dtype: float64

In [24]:
def fill_wind_dir(cols):
    w_9am = cols[0]
    w_3pm = cols[1]
    location = cols[2]
    if (pd.isna(w_9am) == True) and (pd.isna(w_3pm) == False):
        return [w_3pm,w_3pm]
    elif (pd.isna(w_9am) == False) and (pd.isna(w_3pm) == True):
        return [w_9am,w_9am]
    elif (pd.isna(w_9am) == True) and (pd.isna(w_3pm) == True):
        return ['N','N']
    else:
        return [w_9am,w_3pm]
    

In [25]:
df[['WindDir9am','WindDir3pm']] = df[['WindDir9am','WindDir3pm','Location']].apply(fill_wind_dir,axis=1,result_type='expand')

In [26]:
location_wise_speed = df.groupby('Location')[['WindSpeed9am','WindSpeed3pm']].mean()
def fill_wind_speed(cols):
    w_9am = cols[0]
    w_3pm = cols[1]
    location = cols[2]
    if (pd.isna(w_9am) == True) and (pd.isna(w_3pm) == False):
        return [w_3pm,w_3pm]
    elif (pd.isna(w_9am) == False) and (pd.isna(w_3pm) == True):
        return [w_9am,w_9am]
    elif (pd.isna(w_9am) == True) and (pd.isna(w_3pm) == True):
        return list(location_wise_speed.loc[location])
    else:
        return [w_9am,w_3pm]
    

In [27]:
df[['WindSpeed9am','WindSpeed3pm']] = df[['WindSpeed9am','WindSpeed3pm','Location']].apply(fill_wind_speed,axis=1,result_type='expand')

In [28]:
def fill_wind_gust_speed(cols):
    ws_g = cols[0]
    ws_9am = cols[1]
    ws_3pm = cols[2]
    if pd.isna(ws_g) == True:
        if ws_9am >= ws_3pm: 
            return ws_9am
        else:
            return ws_3pm
    else:
        return ws_g

In [29]:
df['WindGustSpeed'] = df[['WindGustSpeed','WindSpeed9am','WindSpeed3pm']].apply(fill_wind_gust_speed,axis=1)

In [30]:
def fill_wind_gust_dir(cols):
    ws_dir = cols[0]
    ws_9am = cols[1]
    ws_3pm = cols[2]
    ws_9am_dir = cols[1]
    ws_3pm_dir = cols[2]
    
    if pd.isna(ws_dir) == True:
        if ws_9am >= ws_3pm: 
            return ws_9am_dir
        else:
            return ws_3pm_dir
    else:
        return ws_dir

In [31]:
df['WindGustDir'] = df[['WindGustDir','WindSpeed9am','WindSpeed3pm','WindDir9am','WindDir3pm']].apply(fill_wind_gust_dir,axis=1,result_type='expand')

In [32]:
(df.isnull().sum().sort_values(ascending=False)*100)/len(df)

Humidity3pm      1.841474
Temp3pm          1.216038
Humidity9am      0.772506
Temp9am          0.164099
MinTemp          0.139330
MaxTemp          0.103723
Date             0.000000
RainToday        0.000000
Pressure3pm      0.000000
Pressure9am      0.000000
WindSpeed9am     0.000000
WindSpeed3pm     0.000000
Location         0.000000
WindDir3pm       0.000000
WindDir9am       0.000000
WindGustSpeed    0.000000
WindGustDir      0.000000
Rainfall         0.000000
RainTomorrow     0.000000
dtype: float64

In [33]:
location_wise_humidity = df.groupby('Location')[['Humidity9am','Humidity3pm']].mean()
def fill_humidity(cols):
    h_9am = cols[0]
    h_3pm = cols[1]
    location = cols[2]
    if (pd.isna(h_9am) == True) and (pd.isna(h_3pm) == False):
        return [h_3pm,h_3pm]
    elif (pd.isna(h_9am) == False) and (pd.isna(h_3pm) == True):
        return [h_9am,h_9am]
    elif (pd.isna(h_9am) == True) and (pd.isna(h_3pm) == True):
        return list(location_wise_humidity.loc[location])
    else:
        return [h_9am,h_3pm]

In [34]:
df[['Humidity9am','Humidity3pm']] = df[['Humidity9am','Humidity3pm','Location']].apply(fill_humidity,axis=1,result_type='expand')

In [35]:
location_wise_temp = df.groupby('Location')[['Temp9am','Temp3pm']].mean()
def fill_temp(cols):
    t_9am = cols[0]
    t_3pm = cols[1]
    location = cols[2]
    if (pd.isna(t_9am) == True) and (pd.isna(t_3pm) == False):
        return [t_3pm,t_3pm]
    elif (pd.isna(t_9am) == False) and (pd.isna(t_3pm) == True):
        return [t_9am,t_9am]
    elif (pd.isna(t_9am) == True) and (pd.isna(t_3pm) == True):
        return list(location_wise_temp.loc[location])
    else:
        return [t_9am,t_3pm]

In [36]:
df[['Temp9am','Temp3pm']] = df[['Temp9am','Temp3pm','Location']].apply(fill_temp,axis=1,result_type='expand')

In [37]:
(df.isnull().sum().sort_values(ascending=False)*100)/len(df)

MinTemp          0.139330
MaxTemp          0.103723
Date             0.000000
Humidity9am      0.000000
RainToday        0.000000
Temp3pm          0.000000
Temp9am          0.000000
Pressure3pm      0.000000
Pressure9am      0.000000
Humidity3pm      0.000000
WindSpeed9am     0.000000
WindSpeed3pm     0.000000
Location         0.000000
WindDir3pm       0.000000
WindDir9am       0.000000
WindGustSpeed    0.000000
WindGustDir      0.000000
Rainfall         0.000000
RainTomorrow     0.000000
dtype: float64

In [38]:
def fill_max_temp(cols):
    temp_9am = cols[0]
    temp_3pm = cols[1]
    max_temp = cols[2]
    if pd.isna(max_temp) == True:
        if temp_9am > temp_3pm:
            return temp_9am
        else:
            return temp_3pm
    else:
        return max_temp
    

In [39]:
df['MaxTemp'] = df[['Temp9am','Temp3pm','MaxTemp']].apply(fill_max_temp,axis=1)

In [40]:
def fill_min_temp(cols):
    temp_9am = cols[0]
    temp_3pm = cols[1]
    min_temp = cols[2]
    if pd.isna(min_temp) == True:
        if temp_9am > temp_3pm:
            return temp_3pm
        else:
            return temp_9am
    else:
        return min_temp
    

In [41]:
df['MinTemp'] = df[['Temp9am','Temp3pm','MinTemp']].apply(fill_min_temp,axis=1)

In [42]:
(df.isnull().sum().sort_values(ascending=False)*100)/len(df)

Date             0.0
WindSpeed3pm     0.0
RainToday        0.0
Temp3pm          0.0
Temp9am          0.0
Pressure3pm      0.0
Pressure9am      0.0
Humidity3pm      0.0
Humidity9am      0.0
WindSpeed9am     0.0
Location         0.0
WindDir3pm       0.0
WindDir9am       0.0
WindGustSpeed    0.0
WindGustDir      0.0
Rainfall         0.0
MaxTemp          0.0
MinTemp          0.0
RainTomorrow     0.0
dtype: float64

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129190 entries, 0 to 145458
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           129190 non-null  object 
 1   Location       129190 non-null  object 
 2   MinTemp        129190 non-null  float64
 3   MaxTemp        129190 non-null  float64
 4   Rainfall       129190 non-null  float64
 5   WindGustDir    129190 non-null  object 
 6   WindGustSpeed  129190 non-null  float64
 7   WindDir9am     129190 non-null  object 
 8   WindDir3pm     129190 non-null  object 
 9   WindSpeed9am   129190 non-null  float64
 10  WindSpeed3pm   129190 non-null  float64
 11  Humidity9am    129190 non-null  float64
 12  Humidity3pm    129190 non-null  float64
 13  Pressure9am    129190 non-null  float64
 14  Pressure3pm    129190 non-null  float64
 15  Temp9am        129190 non-null  float64
 16  Temp3pm        129190 non-null  float64
 17  RainToday      129190 non-null  ob

In [44]:
df['Date'] = pd.to_datetime(df['Date'])

In [45]:
df['Date']

0        2008-12-01
1        2008-12-02
2        2008-12-03
3        2008-12-04
4        2008-12-05
            ...    
145454   2017-06-20
145455   2017-06-21
145456   2017-06-22
145457   2017-06-23
145458   2017-06-24
Name: Date, Length: 129190, dtype: datetime64[ns]

In [46]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

In [47]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,...,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month,Day
0,2008-12-01,Albury,13.4,22.9,0.6,W,44.0,W,WNW,20.0,...,22.0,1007.7,1007.1,16.9,21.8,No,No,2008,12,1
1,2008-12-02,Albury,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,...,25.0,1010.6,1007.8,17.2,24.3,No,No,2008,12,2
2,2008-12-03,Albury,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,...,30.0,1007.6,1008.7,21.0,23.2,No,No,2008,12,3
3,2008-12-04,Albury,9.2,28.0,0.0,NE,24.0,SE,E,11.0,...,16.0,1017.6,1012.8,18.1,26.5,No,No,2008,12,4
4,2008-12-05,Albury,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,...,33.0,1010.8,1006.0,17.8,29.7,No,No,2008,12,5


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129190 entries, 0 to 145458
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           129190 non-null  datetime64[ns]
 1   Location       129190 non-null  object        
 2   MinTemp        129190 non-null  float64       
 3   MaxTemp        129190 non-null  float64       
 4   Rainfall       129190 non-null  float64       
 5   WindGustDir    129190 non-null  object        
 6   WindGustSpeed  129190 non-null  float64       
 7   WindDir9am     129190 non-null  object        
 8   WindDir3pm     129190 non-null  object        
 9   WindSpeed9am   129190 non-null  float64       
 10  WindSpeed3pm   129190 non-null  float64       
 11  Humidity9am    129190 non-null  float64       
 12  Humidity3pm    129190 non-null  float64       
 13  Pressure9am    129190 non-null  float64       
 14  Pressure3pm    129190 non-null  float64       
 15  Temp9

In [49]:
df['Delta_Temp_day'] = abs(df['MaxTemp'] - df['MinTemp'])
df['Delta_Temp_6hr'] = abs(df['Temp9am'] - df['Temp3pm'])

In [50]:
df['Wind_dir_change'] = df['WindDir9am'] != df['WindDir3pm']

In [51]:
df['Wind_dir_change'].value_counts()

Wind_dir_change
True     99320
False    29870
Name: count, dtype: int64

In [52]:
df['Delta_wind_speed_6hr'] = abs(df['WindSpeed9am'] - df['WindSpeed3pm'])

In [53]:
df['Delta_humidity_6hr'] = abs(df['Humidity9am'] - df['Humidity3pm'])

In [54]:
df['Delta_Pressure_6hr'] = abs(df['Pressure9am'] - df['Pressure3pm'])

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129190 entries, 0 to 145458
Data columns (total 28 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Date                  129190 non-null  datetime64[ns]
 1   Location              129190 non-null  object        
 2   MinTemp               129190 non-null  float64       
 3   MaxTemp               129190 non-null  float64       
 4   Rainfall              129190 non-null  float64       
 5   WindGustDir           129190 non-null  object        
 6   WindGustSpeed         129190 non-null  float64       
 7   WindDir9am            129190 non-null  object        
 8   WindDir3pm            129190 non-null  object        
 9   WindSpeed9am          129190 non-null  float64       
 10  WindSpeed3pm          129190 non-null  float64       
 11  Humidity9am           129190 non-null  float64       
 12  Humidity3pm           129190 non-null  float64       
 13  Pres

In [56]:
cat_cols = ['Location','WindGustDir','WindDir9am', 'WindDir3pm','RainToday','Year', 'Month', 'Day','Wind_dir_change']
num_cols = ['MinTemp', 'MaxTemp', 'Rainfall','WindGustSpeed','WindSpeed9am','WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am','Pressure3pm', 'Temp9am', 'Temp3pm','Delta_Temp_day', 'Delta_Temp_6hr','Delta_wind_speed_6hr', 'Delta_humidity_6hr','Delta_Pressure_6hr']
y_col = ['RainTomorrow']

In [57]:
df[num_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
MinTemp,129190.0,12.405136,6.351030,-8.2,7.8,12.2,17.0,39.6
MaxTemp,129190.0,23.404915,6.969160,1.0,18.0,22.8,28.4,48.1
Rainfall,129190.0,2.347090,8.492114,0.0,0.0,0.0,0.8,371.0
WindGustSpeed,129190.0,39.108333,13.974565,0.0,30.0,37.0,46.0,135.0
WindSpeed9am,129190.0,14.246498,8.807017,0.0,7.0,13.0,19.0,87.0
WindSpeed3pm,129190.0,18.986170,8.701856,0.0,13.0,19.0,24.0,87.0
Humidity9am,129190.0,68.527116,18.944177,0.0,57.0,70.0,82.0,100.0
Humidity3pm,129190.0,51.538313,20.668081,0.0,37.0,52.0,66.0,100.0
Pressure9am,129190.0,1017.640463,7.052619,980.5,1013.0,1017.6,1022.3,1041.0
Pressure3pm,129190.0,1015.252408,6.983437,977.1,1010.5,1015.2,1019.9,1039.6


### float 16 has range of 
float16	±0.0000655 to ±65,504

In [58]:
#converting into float 16
for col in num_cols:
    df[col] = df[col].astype('float16')

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129190 entries, 0 to 145458
Data columns (total 28 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Date                  129190 non-null  datetime64[ns]
 1   Location              129190 non-null  object        
 2   MinTemp               129190 non-null  float16       
 3   MaxTemp               129190 non-null  float16       
 4   Rainfall              129190 non-null  float16       
 5   WindGustDir           129190 non-null  object        
 6   WindGustSpeed         129190 non-null  float16       
 7   WindDir9am            129190 non-null  object        
 8   WindDir3pm            129190 non-null  object        
 9   WindSpeed9am          129190 non-null  float16       
 10  WindSpeed3pm          129190 non-null  float16       
 11  Humidity9am           129190 non-null  float16       
 12  Humidity3pm           129190 non-null  float16       
 13  Pres

In [60]:
for col in cat_cols:
    df[col] = df[col].astype('category')

In [61]:
df['RainTomorrow'] = df['RainTomorrow'].apply(lambda x : 1 if x == "Yes" else 0)

In [62]:
df['RainTomorrow'].value_counts()

RainTomorrow
0    100531
1     28659
Name: count, dtype: int64

In [63]:
cats = np.stack([df[col].cat.codes for col in cat_cols],axis=1)
cats = torch.LongTensor(cats)

In [64]:
cats.size()

torch.Size([129190, 9])

In [65]:
conts = np.stack([df[col].values for col in num_cols],axis=1)
conts = torch.Tensor(conts)

In [66]:
conts.shape

torch.Size([129190, 17])

In [67]:
y = torch.tensor(df[y_col].values)

In [68]:
y

tensor([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]])

In [69]:
from sklearn.model_selection import train_test_split

# Convert 'y' to numpy array
y = df[y_col].values

# Split the data with stratification on 'y'
cat_train, cat_test, cont_train, cont_test, y_train, y_test = train_test_split(
    cats, conts, y, test_size=0.25, stratify=y, random_state=42
)

In [70]:
cat_train = torch.LongTensor(cat_train).to(device)
cat_test = torch.LongTensor(cat_test).to(device)
cont_train = torch.Tensor(cont_train).to(device)
cont_test = torch.Tensor(cont_test).to(device)
y_train = torch.Tensor(y_train).to(device)
y_test = torch.Tensor(y_test).to(device)

In [71]:
cat_szs = [len(df[col].cat.categories) for col in cat_cols]
emb_szs = [(size,min(50,(size+1)//2)) for size in cat_szs]
emb_szs

[(45, 23),
 (70, 35),
 (16, 8),
 (16, 8),
 (2, 1),
 (11, 6),
 (12, 6),
 (31, 16),
 (2, 1)]

### Model

In [72]:
class Model(nn.Module):
    
    def __init__(self,emb_szs,n_cont,out_sz,layers,p=0.35):
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(ni,nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(p)
        n_emb = sum([nf for ni,nf in emb_szs])
        
        self.bn_cont = nn.BatchNorm1d(n_cont)
        n_in = n_emb + n_cont
        
        layerlist = []
        for i in layers:
            layerlist.append(nn.Linear(n_in,i))
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in = i
        layerlist.append(nn.Linear(layers[-1],out_sz))
        self.layers = nn.Sequential(*layerlist)
    
    def forward(self,x_cat,x_cont):
        embeddings = []
        for i,e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
        x = torch.cat(embeddings,1)
        x = self.emb_drop(x)
        
        x_cont = self.bn_cont(x_cont)
        x = torch.cat([x,x_cont],1)
        x = self.layers(x)
        return x

In [73]:
model = Model(emb_szs,conts.shape[1],2,[512,256,128,64,32,16,8,4],p=0.4).to(device)

In [74]:
optimizer = optim.Adam(model.parameters(),lr=0.005)
criterion = nn.CrossEntropyLoss()
num_epochs = 1000

In [75]:
losses = []
for epoch in range(num_epochs):
    y_pred = model.forward(cat_train,cont_train)
    loss = criterion(y_pred,y_train.squeeze())
    losses.append(loss.item())
    if (epoch+1)%50 == 0:
        print(f"epoch : {epoch+1}/{num_epochs} loss : {loss.item()}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch : 50/1000 loss : 0.4867715537548065
epoch : 100/1000 loss : 0.38589268922805786
epoch : 150/1000 loss : 0.35543203353881836
epoch : 200/1000 loss : 0.33932092785835266
epoch : 250/1000 loss : 0.32726845145225525
epoch : 300/1000 loss : 0.3199015259742737
epoch : 350/1000 loss : 0.31266355514526367
epoch : 400/1000 loss : 0.30821678042411804
epoch : 450/1000 loss : 0.3045915961265564
epoch : 500/1000 loss : 0.30223265290260315
epoch : 550/1000 loss : 0.2981801927089691
epoch : 600/1000 loss : 0.2941155731678009
epoch : 650/1000 loss : 0.29314735531806946
epoch : 700/1000 loss : 0.28974246978759766
epoch : 750/1000 loss : 0.2894204556941986
epoch : 800/1000 loss : 0.28675734996795654
epoch : 850/1000 loss : 0.28230610489845276
epoch : 900/1000 loss : 0.2828204333782196
epoch : 950/1000 loss : 0.28235912322998047
epoch : 1000/1000 loss : 0.2816915214061737


In [81]:
with torch.no_grad():
    y_eval = model.forward(cat_test,cont_test)

In [82]:
y_test = y_test.cpu().numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'cpu'

In [83]:
y_eval = torch.argmax(y_eval,axis=1).cpu().numpy()

In [84]:
from sklearn.metrics import accuracy_score

In [85]:
print(accuracy_score(y_test,y_eval))

0.8501764815158833
